# Habituation with IBCM neurons gating inhibitory neurons
The details of the model are described in other Jupyter notebooks (e.g., ibcm_inhibition_three_components.ipynb). 

![test](figures/feedforward_inhibitory_network.png)



## Saturation function on IBCM neurons
Here, a mild saturation function is applied to IBCM neurons' activity after lateral inhibitory coupling. So, here, we still have simply
$$ \vec{c} = M\vec{x} $$
but now the inhibited activities are rather
$$ \vec{\bar{c}} = \sigma(LM\vec{x}) $$
where the nonlinear activation function $\sigma$ applies element-wise to $LM \vec{x}$. We choose a function that saturates only when $LM\vec{x}$ is very large, and that is linear for small values of $LM\vec{x}$. In particular, we take
$$ \sigma(u) = s \tanh{\left(\frac{u}{s}\right)} $$
which has derivative
$$ \sigma'(u) = \mathrm{sech}^2{\left(\frac{u}{s}\right)} = 1 - \tanh^2{\left(\frac{u}{s}\right)} = 1 - \left(\frac{\sigma(u)}{s}\right)^2 $$

## Turbulent olfactory environment

Importantly, here, the background used is an approximation of realistic odor concentrations in turbulent environments, based on Celani, Villermaux, Vergassola, 2014. In summary, each odor concentration (sources treated as independent for now) is a succession of whiffs and blanks, with power law distributions of exponent $-3/2$ for the waiting times, and a long-tailed whiff concentration distribution of the form $\sim e^{-c/c_0}/c$. 

Also, of note, we use the Euclidean distance between $\vec{s}$ (the filtered odor, projection neurons layer) and the target odor as a performance metric, instead of the Jaccard similarity of the Kenyon cells layer. The reason: it is easier to work with analytically, easier to understand intuitively, and easier to compute numerically. Anyways, since KC cells implement a locality sensitive hashing, closeness in Euclidean distance should correspond to closeness in neural tags (hashes). 

## Notes on current status of the project (June 2022)
The immediate next step is to test whether new odors can be detected after filtering of this kind of background. 

There are a few steps left towards a fully realistic olfaction model: 
 1. Add (anti)-correlation between odor sources and maybe make the concentration fluctuate during a whiff too. 
 2. Combine odors not linearly but with Gautam's ORN model; this will help saturate concentrations and will also make things less linear, hence further away from what an online PCA could do best. 
 
The backup strategy is to use an alternating background, maybe with blanks, as a proxy for turbulent fluctuations, but a higher chance of success from the IBCM model (which was built for this kind of processes). 

Then, the final step is to compare with other projection learning models, especially online PCA and online ICA.

### Two remarks after playing with the notebook below

1. Saturating the sum of odors with a tanh really helps the model to learn: it prevents the rare, huge concentration events that throw off the averaging. So using Gautam's model on the input layer will really help learn odor components even though it destroys the linearity: linearity is actually a problem when concentrations change too much. 
2. When inhibition between neurons is included, even if we initialize all neurons to pretty large random values, only the neurons with the most extreme values (largest m) converge fast enough; neurons "in the middle" take longer. That's because after inhibition, there is not much left in $\bar{m}$ except for those extreme values. So, a good initial condition would be to start all neurons on $S^n(1)$, the unit hypersphere. 

## Functions of general interest

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sys import byteorder as sys_byteorder

from modelfcts.ibcm import integrate_inhib_ibcm_network_skip_tanh
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    fixedpoint_thirdmoment_perturbtheory
)
from modelfcts.backgrounds import (
    update_powerlaw_times_concs, 
    logof10, 
    sample_background_powerlaw,
    sample_ss_conc_powerlaw, 
    decompose_nonorthogonal_basis, 
    update_alternating_inputs, 
    generate_odorant
)
from modelfcts.tagging import (
    project_neural_tag, create_sparse_proj_mat
)
from utils.statistics import seed_from_gen
from modelfcts.distribs import (
    truncexp1_inverse_transform, 
    truncexp1_density, 
    powerlaw_cutoff_inverse_transform
)
from modelfcts.checktools import check_conc_samples_powerlaw_exp1
from utils.smoothing_function import moving_average
from simulfcts.plotting import (
    plot_cbars_gammas_sums, 
    plot_cbars_gamma_series, 
    plot_3d_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition
)
from utils.metrics import jaccard

### Response to a new odor
This part of the code only runs if the simulation above had ``n_dimensions > n_components``. 

The goal is to see whether a new odor, not linearly dependent of the ones in the background, also gets repressed close to zero, or produces an inhibited output noticeably different from the inhibited background, and still similar to the new odor vector, at least its component perpendicular to the background subspace. 

Need to test for many samples from the background odor distribution. Keep the new odor at a constant concentration, typical of the concentration at which we actually want the system to pick up the new odor. 

I realize that it's fine if the disentanglement of odors isn't perfect at the PN layer: besides the question of habituation, the sparse tag network proposed by Dasgupta does not address too well how multiple odors are disentangled from a complicated mixture. 

In [ ]:
from modelfcts.ibcm import relu_inplace, ibcm_respond_new_odors

In [ ]:
from utils.metrics import l2_norm, l1_norm, linf_norm, cosine_dist

def distance_panel_target(mixes, target):
    """ Compute a panel of distances between the pure (target) new odor and mixtures 
    (which can be without inhibition, with average inhibition, IBCM inhibition, etc.). 
    
    Four distances included, in order: l2, l1, linf, cosine_dist
    
    Args:
        mixes (np.ndarray): mixtures of odors to compute distance from target, 
            the last axis should have the size of target, 
            while other axes are arbitrary.  
        target (np.1darray): target odor vector, same length as
            last axis of mixes. 
    Returns:
        dist_panel (np.ndarray): shape of pure, except the last axis, 
            which has length 4 (for the number of distances computed). 
    """
    # Make axis 0 the axis indexing distance metrics, to begin with
    # And move it to the last axis before returning
    dist_array = np.zeros([4] + list(mixes.shape[:-1]))
    # No need to add axes to target vector; if it is 1d, it is broadcasted
    # along the last axis of mixes, which indexes elements of each vector. 
    dist_array[0] = l2_norm(target - mixes)
    dist_array[1] = l1_norm(target - mixes)
    dist_array[2] = linf_norm(target - mixes)
    dist_array[3] = cosine_dist(target, mixes)
    
    return np.moveaxis(dist_array, 0, -1)

## Complete model: sparse Kenyon cell tags for odors
We need to make new simulations with many more dimensions (ORN types). 

Consequently, to avoid running into memory issues, we only save a subset of time steps in the simulation: this is fine because we are only interested in the slowly-evolving $\vec{w}$ and $\vec{m}$, while we don't care too much for $\vec{x}$'s fast fluctuations. We just want the final average $\vec{w}$ to apply as inhibition to randomly sampled background odors, which we don't even take from simulations but just generate from the steady-state distribution. 

*Modification*: use the Law and Cooper, 1994 trick to improve convergence speed: make the learning rate inversely proportional to the threshold $\Theta$. The IBCM equation (for 1 neuron) gets modified to

$$ \frac{d\vec{m}}{dt} = \frac{\mu}{\Theta} c(c - \Theta) \vec{x}(t) $$

In fact, it works better if we limit the increase of the learning rate, and even more importantly, save the decrease of it only for very late times, once neurons have converged to specific fixed points. The trick is to normalize as follows:

$$ \frac{d\vec{m}}{dt} = \frac{\mu}{\Theta + k_{\Theta}} c(c - \Theta) \vec{x}(t) $$

where $k_{\Theta}$ is relatively large, larger than the value of $\Theta$ when the neurons reach the saddle (?) point where $c_d$ is large but there is no specificity yet. Else, the learning rate freezes too fast and neurons never become specific, they get stuck in that saddle. 

I find that $k_{\Theta} = 5$ is in the sweet spot for this current background, but that depends on the absolute magnitude of the background... Maybe using a separate neuron computing the average $x$ and $m$ to estimate $k_{\Theta}$ would help? But I need to make this scale-independent, it is currently not. 

### Run a new simulation with 25 dimensions

In [ ]:
from modelfcts.ibcm import integrate_inhib_lawibcm_network_skip_tanh

In [ ]:
### General simulation parameters
n_dimensions_tag = 25  # Half the real number for faster simulations
n_neurons_tag = 24
n_components_tag = 6

# Simulation rates and coupling stay the same (try at least)
duration = 320000.0
deltat = 1.0
learnrate_tag = 0.004  # 0.000001 = 1e-6
tau_avg_tag = 800
inhib_rates_tag = [0.00025, 0.00005]  # alpha, beta
# Background components need to be redefined. Extra dimensions are somewhat superfluous
coupling_eta_tag = 0.5/n_neurons_tag
ssat_tag = 50.0
k_c2bar_avg = 2.0
lambd_ibcm = 2.0
ibcm_rates_tag = [learnrate_tag, tau_avg_tag, coupling_eta_tag, lambd_ibcm, ssat_tag, k_c2bar_avg]

use_tanh_sat_tag = False

# Choose symmetric, normalized background odor components
#back_components_tag = np.ones([n_components_tag, n_dimensions_tag]) * 0.2
#for i in range(n_components_tag):
#    back_components_tag[i, i] = 0.8
#    back_components_tag[i] /= np.sqrt(np.sum(back_components_tag[i]**2))

# Choose randomly generated background vectors
rgen_meta_tag = np.random.default_rng(seed=0xf452aff441eb4c4568e97848aa1746b9)  # Good seed
#rgen_meta_tag = np.random.default_rng(seed=0xf45daf0431eb4d4f68e97848aa1746a9)
back_components_tag = np.zeros([n_components_tag, n_dimensions_tag])
for i in range(n_components_tag):
    back_components_tag[i] = generate_odorant(n_dimensions_tag, rgen_meta_tag, lambda_in=0.1)
back_components_tag = back_components_tag / l2_norm(back_components_tag).reshape(-1, 1)
    
# Initial synaptic weights: small positive noise
#init_synapses_tag = 0.1*rgen_meta_tag.random(size=[n_neurons_tag, n_dimensions_tag])
init_synapses_tag = 0.1*rgen_meta_tag.standard_normal(size=[n_neurons_tag, n_dimensions_tag])

# Turbulent background parameters: same rates and constants for all odors
back_params_tag = [
    np.asarray([1.0] * n_components_tag),        # whiff_tmins
    np.asarray([100.] * n_components_tag),       # whiff_tmaxs
    np.asarray([2.0] * n_components_tag),        # blank_tmins
    np.asarray([200.0] * n_components_tag),      # blank_tmaxs
    np.asarray([0.6] * n_components_tag),        # c0s
    np.asarray([0.5] * n_components_tag),        # alphas
]
back_params_tag.append(back_components_tag)

# Initial values of background process variables (t, c for each variable)
init_concs_tag = sample_ss_conc_powerlaw(*back_params_tag[:-1], size=1, rgen=rgen_meta_tag)
init_times_tag = powerlaw_cutoff_inverse_transform(
                rgen_meta_tag.random(size=n_components_tag), *back_params_tag[2:4])
tc_init_tag = np.stack([init_times_tag, init_concs_tag.squeeze()], axis=1)

# Initial background vector 
init_bkvec_tag = tc_init_tag[:, 1].dot(back_components_tag)
# nus are first in the list of initial background params
init_back_list_tag = [tc_init_tag, init_bkvec_tag]

In [ ]:
# Run a heavy simulation
skp_tag = 20
update_fct = update_powerlaw_times_concs_saturate if use_tanh_sat_tag else update_powerlaw_times_concs
sim_results = integrate_inhib_lawibcm_network_skip_tanh(init_synapses_tag, update_fct, 
                    init_back_list_tag, ibcm_rates_tag, inhib_rates_tag, back_params_tag, duration, deltat, 
                    seed=seed_from_gen(rgen_meta_tag), noisetype="uniform", skp=skp_tag)
tser_tag, nuser_tag, bkvecser_tag, mser_tag, cbarser_tag, thetaser_tag, wser_tag, yser_tag = sim_results

### Check the output a bit

In [ ]:
# Calculate cgammas_bar and mbars
transient = 100000 // skp_tag
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser_tag = mser_tag*(1.0 + coupling_eta_tag) - coupling_eta_tag*np.sum(mser_tag, axis=1, keepdims=True)
c_gammas_tag = mser_tag.dot(back_components_tag.T)
cbars_gamma_tag = mbarser_tag.dot(back_components_tag.T)

sums_cbars_gamma_tag = np.sum(cbars_gamma_tag, axis=2)
sums_cbars_gamma2_tag = np.sum(cbars_gamma_tag*cbars_gamma_tag, axis=2)

# Analytical prediction, exact: need moments of nu. Easiest to compute numerically. 
# Although could do analytically by conditioning on puff==on or off
conc_ser_tag = nuser_tag[:, :, 1]
# Odors are all iid so we can average over all odors
mean_conc = np.mean(conc_ser_tag)
sigma2_conc = np.var(conc_ser_tag)
thirdmom_conc = np.mean((conc_ser_tag - mean_conc)**3)
moments_conc = [mean_conc, sigma2_conc, thirdmom_conc]

# Analytical prediction
res = fixedpoint_thirdmoment_exact(moments_conc, 1, n_components_tag-1, lambd=lambd_ibcm)
c_specif, c_nonspecif = res[:2]

# Constaint 1: sum of c_gammas for each neuron
#print("Comparison to analytical fixed points")
#print("This should be all ones:", np.mean(sums_cbars_gamma_sym[transient:], axis=0) * averages_nu.mean())
# Constraint 2: sum of c_gammas^2 for each neuron, copmared to 1/sigma^2
#print("This should be all zeros:", np.mean(sums_cbars_gamma2_sym[transient:], axis=0)*sigma2 - 1.0)

# Count how many components are at each possible value. Use cbar = 2.0 as a split. 
split_val = 2.0
cbars_gamma_mean_tag = np.mean(cbars_gamma_tag[transient:], axis=0)
cgammas_bar_counts = {"above": int(np.sum(cbars_gamma_mean_tag.flatten() > split_val)), 
                      "below": int(np.sum(cbars_gamma_mean_tag.flatten() <= split_val))}
print(cgammas_bar_counts)

In [ ]:
print("Late mean:", np.mean(cbarser_tag[transient:]**2, axis=0))
print("Early mean:", np.mean(cbarser_tag[:3]**2, axis=0))

In [ ]:
# Plot the cbar2_avg term throughout
cbar2_avg_ser = moving_average(cbarser_tag*cbarser_tag, kernelsize=tau_avg_tag)
neurons_cmap = sns.color_palette("Greys", n_colors=n_neurons_tag)
fig, ax = plt.subplots()
for i in range(n_neurons_tag):
    ax.plot(tser_tag[:-tau_avg_tag], cbar2_avg_ser[:-tau_avg_tag, i], color=neurons_cmap[i])
ax.set(xlabel="Time (x1000)", ylabel=r"$\bar{c}^2$ moving average")
plt.show()
plt.close()

In [ ]:
fig , ax, _ = plot_cbars_gamma_series(tser_tag, cbars_gamma_tag, skp=10, transient=300000 // skp_tag)
# Compare to exact analytical fixed point solution
ax.axhline(c_specif, ls="--", color="grey", label=r"Analytical $\bar{c}_{\gamma=\mathrm{specific}}$")
ax.axhline(c_nonspecif, ls="--", color="grey", label=r"Analytical $\bar{c}_{\gamma=\mathrm{non}}$")
plt.show()
plt.close()

In [ ]:
# Correlation between nu's and c's, see if some neurons are specific to odors
# Each neuron turns out to correlate its response to  one concentration
# that means it is specific to that odor. 
#mser_norm_centered = l2_norm(mser_tag, axis=-1) - np.mean(l2_norm(mser_tag[transient:], axis=-1), axis=0)
cbarser_norm_centered = cbarser_tag - np.mean(cbarser_tag[transient:], axis=0)
conc_ser_centered = nuser_tag[:, :, 1] - np.mean(nuser_tag[transient:, :, 1], axis=0)
correl_c_nu = np.mean(cbarser_norm_centered[transient:, :, None] * conc_ser_centered[transient:, None, :], axis=0)
fig, ax = plt.subplots()
img = ax.imshow(correl_c_nu.T)
ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
             r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
            location="top")
plt.show()
plt.close()

# Check if each component has at least one neuron
for comp in range(n_components_tag):
    print("Number of neurons specific to component {}: {}".format(
                        comp, np.sum(np.mean(cbars_gamma_tag[-2000:, :, comp], axis=0) > 2.0)))

#### Remark
The above looks quite promising: each IBCM neuron is specific to one odor. Problem is that apparently, it happens often that one of the odors has no specific neuron. Of course it's not going to inhibit well in this case... 

In [ ]:
fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(tser_tag, bkvecser_tag, yser_tag, skp=10)

# Compute noise reduction factor, annotate
transient = 100000 // skp_tag
avg_bknorm = np.mean(bknorm_ser[transient:])
avg_ynorm = np.mean(ynorm_ser[transient:])
avg_reduction_factor = avg_ynorm / avg_bknorm
std_bknorm = np.std(bknorm_ser[transient:])
std_ynorm = np.std(ynorm_ser[transient:])
std_reduction_factor = std_ynorm / std_bknorm

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(avg_reduction_factor * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(std_reduction_factor * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(std_reduction_factor * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

### Compute and compare projection tags after inhibition

In [ ]:
### New odor, mix, and inhibit
### Repeat for many new odors (and ideally, should repeat for many backgrounds)
### But for now, assume all simulations would give similarly good inhibition. 
n_test_new_odors = 100
mix_frac = 0.5

# Average m and w with which we will inhibit
transient_tag = 250000 // skp_tag
mtag_mean = np.mean(mser_tag[transient_tag:], axis=0)
wtag_mean = np.mean(wser_tag[transient_tag:], axis=0)

# Background samples, valid for all new test odors
back_samples_tag, _ = sample_background_powerlaw(back_components_tag, *back_params_tag[:-1], 
                                          size=100, rgen=rgen_meta_tag)
inhib_ibcm_samples_tag = []
inhib_avg_samples_tag = []
mix_samples_tag = []
new_odor_targets = []
for i in range(n_test_new_odors):
    # New odor
    #new_odor_tag = np.roll(back_components_tag[0], shift=-1)  # Should be a new vector
    new_odor_tag = generate_odorant(n_dimensions_tag, rgen_meta_tag)
    new_odor_tag = new_odor_tag / l2_norm(new_odor_tag)
    new_odor_targets.append(new_odor_tag)
    
    # Background samples, then add new odor
    typical_conc = np.mean(back_params_tag[-3]) * np.mean(back_params_tag[-2])
    mix_samples = back_samples_tag + mix_frac * new_odor_tag.reshape(1, -1) * typical_conc
    if use_tanh_sat_tag:
        mix_samples = 3.0 * np.tanh(mix_samples / 3.0)
    mix_samples_tag.append(mix_samples)

    # Compare to inhibition of the average background, followed by ReLU
    if use_tanh_sat_tag:
        avg_back = np.mean(3.0 * np.tanh(bkvecser_tag / 3.0), axis=0)
    else:
        avg_back = np.mean(bkvecser_tag, axis=0)
    a_over_ab = inhib_rates_tag[0] / sum(inhib_rates_tag)
    inhib_avg_samples = relu_inplace(mix_samples - a_over_ab * avg_back.reshape(1, -1)) + 0.0001
    inhib_avg_samples_tag.append(inhib_avg_samples)

    # Inhibition of each generated sample and statistics on performance
    inhib_ibcm_samples = ibcm_respond_new_odors(mix_samples, mtag_mean, wtag_mean, ibcm_rates_tag)
    inhib_ibcm_samples_tag.append(inhib_ibcm_samples)

mix_samples_tag = np.asarray(mix_samples_tag)
inhib_avg_samples_tag = np.asarray(inhib_avg_samples_tag)
inhib_ibcm_samples_tag = np.asarray(inhib_ibcm_samples_tag)
new_odor_targets = np.asarray(new_odor_targets)

In [ ]:
# Compute tags. This won't be great because too few dimensions to begin with, but try anyways. 
projtag_kwargs = dict(kc_sparsity=0.05, adapt_kc=True, n_pn_per_kc=3, fix_thresh=None,
                      project_thresh_fact=0.05)
proj_mat = create_sparse_proj_mat(n_kc=int(2000/50*n_dimensions_tag), n_rec=n_dimensions_tag, 
                        rgen=rgen_meta_tag, fraction_filled=projtag_kwargs["n_pn_per_kc"]/n_dimensions_tag)

In [ ]:
# Compute tags and Jaccard distances between target odor and mixture without or with inhibition
jaccards_inhib_none = []
jaccards_inhib_avg = []
jaccards_inhib_ibcm = []
for i in range(mix_samples_tag.shape[0]):
    target_tag = project_neural_tag(new_odor_targets[i], new_odor_targets[i], proj_mat, **projtag_kwargs)
    for j in range(mix_samples_tag.shape[1]):
        tag_none = project_neural_tag(mix_samples_tag[i, j], mix_samples_tag[i, j], proj_mat, **projtag_kwargs)
        tag_avg = project_neural_tag(inhib_avg_samples_tag[i, j], mix_samples_tag[i, j], proj_mat, **projtag_kwargs)
        tag_ibcm = project_neural_tag(inhib_ibcm_samples_tag[i, j], mix_samples_tag[i, j], proj_mat, **projtag_kwargs)
        jaccards_inhib_none.append(jaccard(target_tag, tag_none))
        jaccards_inhib_avg.append(jaccard(target_tag, tag_avg))
        jaccards_inhib_ibcm.append(jaccard(target_tag, tag_ibcm))


In [ ]:
# Histograms of Jaccard similarities: larger similarity is better
fig, ax = plt.subplots()
clr_none = "xkcd:navy blue"
clr_ibcm = "xkcd:turquoise"
clr_avg = "xkcd:orangey brown"

ax.hist(jaccards_inhib_none, label="No inhibition", facecolor=clr_none, alpha=0.6, 
        edgecolor=clr_none, density=True)
ax.axvline(np.median(jaccards_inhib_none), color=clr_none, ls="--", lw=1.0)
ax.hist(jaccards_inhib_avg, label="Average inhibition", facecolor=clr_avg, alpha=0.6, 
        edgecolor=clr_avg, density=True)
ax.axvline(np.median(jaccards_inhib_avg), color=clr_avg, ls="--", lw=1.0)
ax.hist(jaccards_inhib_ibcm, label="IBCM inhibition", facecolor=clr_ibcm, alpha=0.6, 
        edgecolor=clr_ibcm, density=True)
ax.axvline(np.median(jaccards_inhib_ibcm), color=clr_ibcm, ls="--", lw=1.0)

ax.set(xlabel="Jaccard similarity", ylabel="Probability density", title="Jaccard similarity (higher is better)")
ax.legend()
fig.tight_layout()
do_save = False
if mix_frac == 0.2 and do_save:
    fig.savefig("figures/detection/jaccard_similarity_ibcm_average_none_f20percent.pdf", transparent=True)
elif mix_frac == 0.5 and do_save:
    fig.savefig("figures/detection/jaccard_similarity_ibcm_average_none_f50percent.pdf", transparent=True)
plt.show()
plt.close()

In [ ]:
# Are jaccard tags of inhibited background odors going towards empty?
# Check a few tags
tag_series = []
norm_series = []
for it in range(0, tser_tag.shape[0], 200):
    yvec = yser_tag[it]
    tag_series.append(len(project_neural_tag(yser_tag[it], bkvecser_tag[it], proj_mat, **projtag_kwargs)))
    norm_series.append(l2_norm(bkvecser_tag[it])*20)
fig, ax = plt.subplots()
ax.plot(range(len(tag_series)), tag_series, marker="o")
ax.plot(range(len(tag_series)), norm_series, marker='s')
ax.set(xlabel="Time step (x20 00)", ylabel="Background tag size")
plt.show()
plt.close()

### Check background statistics

In [ ]:
fig, ax = plt.subplots()
t_window = slice(0, 2000 //skp_tag, 1)
for i in range(n_components_tag):
    ax.plot(tser_tag[t_window]/1000, nuser_tag[t_window, i, 1], label="Odor {}".format(i))
ax.legend()
ax.set(xlabel="Time (x1000 steps)", ylabel="Concentration")
fig.set_size_inches(4., 2.25)
fig.tight_layout()
#fig.savefig("figures/powerlaw/sample_background_concentrations_time_series.pdf", transparent=True, 
#           bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Check that the background process matches the intended distribution
fig, axes = check_conc_samples_powerlaw_exp1(nuser_tag[:, :, 1].T, *back_params_tag[:-1])
plt.show()
plt.close()

### Conclusion
None of the models work very well on turbulent background, even with tanh saturation of ORNs. In fact, for some cases, the new odor is better detected without any inhibition. IBCM only gives a marginally better median, nothing to brag about. Need to find a way to make it converge faster to a useful fixed point, to make neurons distribute themselves better, and to make the $\vec{m}$ fluctuate less. 

Note: even without ORN saturation, it can work as well as with it, as long as the run and IBCM parameters are chosen well enough to ensure convergence. 

### Remarks
1. We should also compute Jaccard similarity to background odors: if there is a background odor more similar, we are really failing the test. The proper metric should be the ratio of Jaccard(target)/Jaccard(most similar background). 
2. Instead of using the average $\vec{m}$ and $\vec{w}$, I should. Maybe this will be worse, if fluctuations of $\vec{m}$ are detrimental. But maybe it will be better, if the $\vec{w}$ track these fluctuations well enough and compensate for them. 
3. We should use multiple different background simulations, instead of just one like I did here. 
4. Pray for PCA to fare worse... 

So, once I have all my models working well enough, then I should subject all of them to this longer testing. 

# Save some results for further plotting
In particular, save excerpt of concentration time series. 

# Other tests

In [ ]:
raise NotImplementedError("I have not adapted the code below this point.")

# Comparison to ideal inhibitory network
In a linear algebra perspective, the best inhibition that could possibly be achieved of a new odor plus background mixture is that the whole component of the new odor parallel to the vector subspace spanned by the background odors is suppressed, while the component perpendicular to it is kept. Indeed, the appearance of the new odor's component in the background space cannot be distinguished from a fluctuation of the background (unless we had neurons tracking statistics of typical activations in that space, but not obvious how to get that). At any rate, this is the best we can hope our IBCM inhibition network will achieve. 

In [ ]:
def find_projector(a):
    """ Calculate projector a a^+, which projects
    a column vector on the vector space spanned by columns of a. 
    """
    a_inv = np.linalg.pinv(a)
    return a.dot(a_inv)
    
def find_parallel_component(x, basis, projector=None):
    """
    Args:
        x (np.ndarray): 1d array of length D containing the vector to decompose. 
        basis (np.ndarray): 2d matrix of size DxK where each column is one
            of the linearly independent background vectors. 
        projector (np.ndarray): 2d matrix A A^+, the projector on the vector
            space spanned by columns of basis. 
    Return:
        x_par (np.ndarray): component of x found in the vector space of basis
            The perpendicular component can be obtained as x - x_par. 
    """
    # If the projector is not provided yet
    if projector is None:
        # Compute Moore-Penrose pseudo-inverse and AA^+ projector
        projector = find_projector(basis)
    x_par = projector.dot(x)
    return x_par

def ideal_linear_inhibitor(x_n_par, x_n_ort, x_back, f, alpha, beta):
    """ Calculate the ideal projection neuron layer, which assumes
    perfect inhibition (down to beta/(alpha+beta)) of the component of the mixture
    parallel to the background odors' vector space, while leaving the orthogonal
    component of the new odor untouched. 
    
    Args:
        x_n_par (np.1darray): new odor, component parallel to background vector space
        x_n_ort (np.1darray): new odor, component orthogonal to background vector space 
        x_back (np.2darray): background samples, one per row
        f (float): mixture fraction (hard case is f=0.2)
        alpha (float): inhibitory weights learning rate alpha
        beta (float): inhibitory weights decaying rate beta
    
    Returns:
        s (np.1darray): projection neurons after perfect linear inhibition
    """
    # Allow broadcasting for multiple x_back vectors
    factor = beta / (alpha + beta)
    s = factor * f*x_n_par + f*x_n_ort
    # I thought the following would have been even better, but turns out it is worse for small f
    #s = f*x_n_par + f*x_n_ort
    s = s.reshape(1, -1) + factor * (1.0-f) * x_back
    return s

In [ ]:
# Reuse each new odor in new_odor_targets and each background in back_samples_tag
# Compute the projector on the background odor components only once
# Compute parallel component of each new odor
# Mix it with all background samples at once using broadcasting capability of ideal_linear_inhibitor function
background_projector = find_projector(back_components_tag.T)
inhib_ideal_samples_tag = []
for od in new_odor_targets:
    # Decompose
    od_par = find_parallel_component(od, basis=back_components_tag.T, projector=background_projector)
    od_ort = od - od_par
    # Compute the perfectly inhibited mixture with each background sample
    inhib_ideal = ideal_linear_inhibitor(od_par, od_ort, back_samples_tag, mix_frac, *inhib_rates)
    # Background reduced to b/(a+b), new odor intact? Perfect inhibition
    #inhib_ideal = inhib_rates[1] / sum(inhib_rates) * (1.0 - mix_frac) * back_samples_tag + mix_frac * od.reshape(1, -1)
    inhib_ideal_samples_tag.append(inhib_ideal)
inhib_ideal_samples_tag = np.asarray(inhib_ideal_samples_tag)

# Compute neural tags of the ideal inhibited mixtures and compare to target tags. 
# Compute tags and Jaccard distances between target odor and mixture without or with inhibition
jaccards_inhib_ideal = []
for i in range(new_odor_targets.shape[0]):
    target_tag = project_neural_tag(new_odor_targets[i], new_odor_targets[i], proj_mat, **projtag_kwargs)
    for j in range(back_samples_tag.shape[0]):
        mix_sample_tag = back_samples_tag[j] + mix_frac * new_odor_targets[i]
        tag_ideal = project_neural_tag(inhib_ideal_samples_tag[i, j], mix_sample_tag, proj_mat, **projtag_kwargs)
        jaccards_inhib_ideal.append(jaccard(target_tag, tag_ideal))


In [ ]:
# Histograms of Jaccard similarities: larger similarity is better
fig, ax = plt.subplots()
clr_map = {"none": "xkcd:navy blue", "average": "xkcd:orangey brown", 
           "ibcm":"xkcd:turquoise", "ideal": "xkcd:powder blue", "ideal2":"xkcd:pale rose"}

ax.hist(jaccards_inhib_ideal, label="Ideal inhibition", facecolor=clr_map["ideal"], alpha=0.6, 
        edgecolor=clr_map["ideal"], density=True)
ax.axvline(np.median(jaccards_inhib_ideal), color=clr_map["ideal"], ls="--", lw=1.0)
ax.hist(jaccards_inhib_avg, label="Average inhibition", facecolor=clr_map["average"], alpha=0.6, 
        edgecolor=clr_map["average"], density=True)
ax.axvline(np.median(jaccards_inhib_avg), color=clr_map["average"], ls="--", lw=1.0)
ax.hist(jaccards_inhib_ibcm, label="IBCM inhibition", facecolor=clr_map["ibcm"], alpha=0.6, 
        edgecolor=clr_map["ibcm"], density=True)
ax.axvline(np.median(jaccards_inhib_ibcm), color=clr_map["ibcm"], ls="--", lw=1.0)

ax.set(xlabel="Jaccard similarity", ylabel="Probability density", title="Jaccard similarity (higher is better)")
ax.legend()
fig.tight_layout()
plt.show()
plt.close()

# Performance as a function of f
I expect to see a relatively sharp drop of median performance for IBCM a bit above $f=\beta/(\alpha + \beta)$, and at this value for the ideal inhibition. 

In [ ]:
def compute_median_performances(back_samples, new_odors, f, projmat, proj_kwargs, 
                                m_mean, w_mean, eta, inhib_ab, back_components):
    """ Compute median Jaccard similarity for the different inhibition methods we have, 
    for a given value of mixture parameter f. """
    all_jaccard_pairs_dict = {"none":[], "average":[], "ibcm":[], "ideal":[], "ideal2":[]}  # list of lists, one per method
    back_proj = find_projector(back_components.T)
    for i in range(new_odors.shape[0]):
        # Compute target tag
        target_tag = project_neural_tag(new_odors[i], new_odors[i], projmat, **proj_kwargs)
        # Prepare mixtures
        mix_samples = back_samples*(1.0 - f) + new_odors[i:i+1]*f
        
        # Compute inhibited mixtures with the different methods
        # No inhibition: just use mix_samples
        # Average inhibition
        avg_back_tag = averages_nu.dot(back_components_tag)
        a_over_ab = inhib_ab[0] / sum(inhib_ab)
        inhib_avg_samples = mix_samples - a_over_ab * avg_back_tag.reshape(1, -1)

        # Inhibition of each generated sample and statistics on performance
        inhib_ibcm_samples = ibcm_respond_new_odors(mix_samples, m_mean, w_mean, eta)
        
        # Ideal inhibition
        od_par = find_parallel_component(new_odors[i], basis=back_components.T, projector=back_proj)
        od_ort = new_odors[i] - od_par
        # Compute the perfectly inhibited mixture with each background sample
        inhib_ideal_samples = ideal_linear_inhibitor(od_par, od_ort, back_samples, f, *inhib_ab)
        # Background reduced to b/(a+b), new odor intact?
        inhib_ideal2_samples = (1.0 - a_over_ab) * (1.0 - f) * back_samples + f * new_odors[i:i+1]
    
        # For each inhibited mixture, compute jaccard similarity
        current_jaccard_dict = {"none":[], "average":[], "ibcm":[], "ideal":[], "ideal2":[]}
        for j in range(back_samples.shape[0]):
            mix_tag = project_neural_tag(mix_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["none"].append(jaccard(target_tag, mix_tag))
            
            # Average
            mix_tag = project_neural_tag(inhib_avg_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["average"].append(jaccard(target_tag, mix_tag))
        
            # IBCM
            mix_tag = project_neural_tag(inhib_ibcm_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["ibcm"].append(jaccard(target_tag, mix_tag))
            
            # Ideal
            mix_tag = project_neural_tag(inhib_ideal_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["ideal"].append(jaccard(target_tag, mix_tag))
            
            # Ideal 2
            mix_tag = project_neural_tag(inhib_ideal2_samples[j], mix_samples[j], projmat, **proj_kwargs)
            current_jaccard_dict["ideal2"].append(jaccard(target_tag, mix_tag))
            
        # Add those values to the total list
        for method in all_jaccard_pairs_dict.keys():
            all_jaccard_pairs_dict[method].append(current_jaccard_dict[method])
    
    # Convert to 2d array and compute median
    # Could choose to have one median per odor or per background sample
    all_jaccard_pairs_dict = {k:np.asarray(a) for k, a in all_jaccard_pairs_dict.items()}
    median_jaccard_pairs_dict = {k:np.median(a) for k, a in all_jaccard_pairs_dict.items()}
    return median_jaccard_pairs_dict
    

In [ ]:
# Use previous functions for various f values
median_jaccards = {"none":[], "average":[], "ibcm":[], "ideal":[], "ideal2":[]}
f_range = np.arange(0.1, 0.8, 0.1)
for f in f_range:
    meds = compute_median_performances(back_samples_tag, new_odor_targets, f, proj_mat, projtag_kwargs, 
                                mtag_mean, wtag_mean, coupling_eta, inhib_rates, back_components_tag)
    for k in meds:
        median_jaccards[k].append(meds[k])
    print("Done f = {:.2f}".format(f))

In [ ]:
fig, ax = plt.subplots()
labelmap = {"none":"None", "average":"Average", "ibcm":"IBCM", "ideal":r"Ideal $\perp$", "ideal2":"Ideal all"}
for k in median_jaccards:
    ax.plot(f_range, median_jaccards[k], color=clr_map[k], label=labelmap[k], lw=3)
ax.set(xlabel="Fraction $f$ of new odor", ylabel="Median Jaccard similarity")
ax.legend(title="Inhibition method")
fig.set_size_inches(4, 3)
fig.tight_layout()
#fig.savefig("figures/detection/inhibition_jaccard_comparison_methods.pdf", transparent=True)
plt.show()
plt.close()